<a href="https://colab.research.google.com/github/psaw/hse-ai24-ml/blob/main/Chocolate_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline-решение

## Импорт библиотек, загрузка данных

In [94]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [3]:
TRAIN = "https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/chocolate_train.csv"
TEST = "https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/chocolate_test_new.csv"

In [95]:
train_df = pd.read_csv(TRAIN)

## Обзор данных, подготовка к обучению

In [96]:
train_df.head()

,Company,Specific Bean Origin,REF,Review,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin
0,Willie's Cacao,Rio Caribe,457,2009,72%,U.K.,3.25,Trinitario,Venezuela
1,Beschle (Felchlin),"Ocumare, Premier Cru, Quizas No. 2",508,2010,72%,Switzerland,3.50,,Venezuela
2,Dark Forest,Tanzania,1554,2015,70%,U.S.A.,3.00,,Tanzania
3,Brasstown aka It's Chocolate,Cooproagro,1125,2013,72%,U.S.A.,3.00,Trinitario,Dominican Republic
4,Pralus,"Java, Indonesie",32,2006,75%,France,3.50,Criollo,Indonesia


Удаляем все строки с пропусками

In [97]:
train_df.dropna(axis=0, inplace=True)

Выделяем матрицу объект-признак и вектор с целевой переменной

In [124]:
X_train = train_df.drop('Rating', axis=1)
y_train = train_df['Rating']

## Преобразование данных

In [127]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1253 entries, 0 to 1254
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Company               1253 non-null   object
 1   Specific Bean Origin  1253 non-null   object
 2   REF                   1253 non-null   int64 
 3   Review                1253 non-null   int64 
 4   Cocoa Percent         1253 non-null   object
 5   Company Location      1253 non-null   object
 6   Bean Type             1253 non-null   object
 7   Broad Bean Origin     1253 non-null   object
dtypes: int64(2), object(6)
memory usage: 120.4+ KB


In [170]:
def cocoa_percent_transformer(X):
    X['Cocoa Percent_numeric'] = X['Cocoa Percent'].str.rstrip('%').astype(float) / 100.0
    return X


def split_and_replace(X, column_name, separator, replace_value):
    X[column_name+'_cat'] = X[column_name].str.split(separator, expand=True)[0].replace(replace_value, np.nan)
    return X


def clean_df(df):
    # Применение ColumnTransformer к данным
    _df = cocoa_percent_transformer(df)
    print("1: ", _df.shape)
    _df = split_and_replace(_df, 'Broad Bean Origin', ',', '\xa0')
    print("2: ", _df.shape)
    _df = split_and_replace(_df, 'Specific Bean Origin', ',', '\xa0')
    print("3: ", _df.shape)
    _df = split_and_replace(_df, 'Bean Type', r'[ ,(]', '\xa0')
    print("4: ", _df.shape)
    # _df['Cocoa Percent'] = _df['Cocoa Percent'].astype('float64')
    print("5: ", _df.shape)
    # Объединение с оригинальными данными, за исключением удаляемых столбцов
    _df = _df.drop(['REF', 'Cocoa Percent', 'Bean Type', 'Broad Bean Origin', 'Specific Bean Origin'], axis=1)
    print("6: ", _df.shape)
    return _df

print("in: ", X_train.shape)
X_train_transformed = clean_df(X_train.copy())
print("out: ", X_train_transformed.shape)
X_train_transformed

in:  (1253, 8)
1:  (1253, 9)
2:  (1253, 10)
3:  (1253, 11)
4:  (1253, 12)
5:  (1253, 12)
6:  (1253, 7)
out:  (1253, 7)


,Company,Review,Company Location,Cocoa Percent_numeric,Broad Bean Origin_cat,Specific Bean Origin_cat,Bean Type_cat
0,Willie's Cacao,2009,U.K.,0.72,Venezuela,Rio Caribe,Trinitario
1,Beschle (Felchlin),2010,Switzerland,0.72,Venezuela,Ocumare,NaN
2,Dark Forest,2015,U.S.A.,0.70,Tanzania,Tanzania,NaN
3,Brasstown aka It's Chocolate,2013,U.S.A.,0.72,Dominican Republic,Cooproagro,Trinitario
4,Pralus,2006,France,0.75,Indonesia,Java,Criollo
...,...,...,...,...,...,...,...
1250,Artisan du Chocolat,2009,U.K.,0.80,Madagascar,Madagascar,Criollo
1251,Marana,2016,Peru,0.70,Peru,Cusco,NaN
1252,Arete,2015,U.S.A.,0.68,Peru,Nacional,Forastero
1253,Fresco,2011,U.S.A.,0.72,Dominican Republic,Conacado,NaN


In [167]:
X_train_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1253 entries, 0 to 1254
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Company                   1253 non-null   object 
 1   Review                    1253 non-null   int64  
 2   Company Location          1253 non-null   object 
 3   Cocoa Percent_numeric     1253 non-null   float64
 4   Broad Bean Origin_cat     1198 non-null   object 
 5   Specific Bean Origin_cat  1253 non-null   object 
 6   Bean Type_cat             625 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 110.6+ KB


### Заполнение пропусков

In [185]:
X_train_transformed.isna().sum()

Company                     0
Review                      0
Company Location            0
Cocoa Percent_numeric       0
Broad Bean Origin_cat       0
Specific Bean Origin_cat    0
Bean Type_cat               0
dtype: int64

In [184]:
# Заполнение пропусков в Bean Type_cat на основе группировки по Broad Bean Origin_cat и Specific Bean Origin_cat
X_train_transformed['Bean Type_cat'] = X_train_transformed.groupby(['Broad Bean Origin_cat', 'Specific Bean Origin_cat'])['Bean Type_cat'].transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
)

X_train_transformed['Bean Type_cat'] = X_train_transformed.groupby(['Broad Bean Origin_cat'])['Bean Type_cat'].transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
)

X_train_transformed['Bean Type_cat'] = X_train_transformed.groupby(['Specific Bean Origin_cat'])['Bean Type_cat'].transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
)

X_train_transformed['Bean Type_cat'].fillna('unknown', inplace=True)

/var/folders/zc/p2mfk5gx5f1cf_g75s52h3940000gn/T/ipykernel_9158/3169443450.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
/var/folders/zc/p2mfk5gx5f1cf_g75s52h3940000gn/T/ipykernel_9158/3169443450.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
/var/folders/zc/p2mfk5gx5f1cf_g75s52h3940000gn/T/ipykernel_9158/3169443450.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is depre

In [174]:
# Альтернативный вариант с использованием fillna
def impute_by_group_fillna(df, group_col, target_col):
    '''
    Заполняет пропущенные значения в target_col на основе group_col
    используя mode() для каждой группы
    '''
    return df.groupby(group_col)[target_col].transform(
        lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
    )

# Применяем альтернативный вариант
X_train_transformed['Broad Bean Origin_cat'] = impute_by_group_fillna(
    X_train_transformed, 
    'Specific Bean Origin_cat', 
    'Broad Bean Origin_cat'
)

# Заполнение всех пустых значений Broad Bean Origin_cat значением Specific Bean Origin_cat
X_train_transformed['Broad Bean Origin_cat'].fillna(X_train_transformed['Specific Bean Origin_cat'], inplace=True)


/var/folders/zc/p2mfk5gx5f1cf_g75s52h3940000gn/T/ipykernel_9158/4033484276.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
/var/folders/zc/p2mfk5gx5f1cf_g75s52h3940000gn/T/ipykernel_9158/4033484276.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the origina

## Обучение модели

Устанавливаем катбуст и обучаем его с гиперпараметрами по умолчанию на всех данных

In [186]:
X_train_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1253 entries, 0 to 1254
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Company                   1253 non-null   object 
 1   Review                    1253 non-null   int64  
 2   Company Location          1253 non-null   object 
 3   Cocoa Percent_numeric     1253 non-null   float64
 4   Broad Bean Origin_cat     1253 non-null   object 
 5   Specific Bean Origin_cat  1253 non-null   object 
 6   Bean Type_cat             1253 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 110.6+ KB


In [187]:
cat_features = [0, 2, 4, 5, 6]

In [188]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(cat_features=cat_features)

In [189]:
model.fit(X_train_transformed, y_train)

Learning rate set to 0.042428
0:	learn: 0.4760830	total: 66.3ms	remaining: 1m 6s
1:	learn: 0.4736058	total: 68.5ms	remaining: 34.2s
2:	learn: 0.4717317	total: 69.4ms	remaining: 23.1s
3:	learn: 0.4695353	total: 71.1ms	remaining: 17.7s
4:	learn: 0.4673991	total: 74.3ms	remaining: 14.8s
5:	learn: 0.4655590	total: 75.6ms	remaining: 12.5s
6:	learn: 0.4638200	total: 76.7ms	remaining: 10.9s
7:	learn: 0.4617710	total: 79.1ms	remaining: 9.81s
8:	learn: 0.4603815	total: 80.3ms	remaining: 8.85s
9:	learn: 0.4590599	total: 82.2ms	remaining: 8.14s
10:	learn: 0.4578225	total: 84.3ms	remaining: 7.57s
11:	learn: 0.4565816	total: 85.4ms	remaining: 7.03s
12:	learn: 0.4561105	total: 85.7ms	remaining: 6.51s
13:	learn: 0.4545640	total: 87.3ms	remaining: 6.14s
14:	learn: 0.4527627	total: 89.3ms	remaining: 5.87s
15:	learn: 0.4516164	total: 90.9ms	remaining: 5.59s
16:	learn: 0.4496930	total: 92.3ms	remaining: 5.34s
17:	learn: 0.4482377	total: 93.6ms	remaining: 5.11s
18:	learn: 0.4469769	total: 95.3ms	remaining

In [191]:
model.score(X_train_transformed, y_train)

0.5378868188838823

In [ ]:
# TODO: что можно сделать для улучшения модели:
# 1. Масштабирование признаков
# 2. Заполнение пропусков 
# 3. Удаление выбросов
# 4. Подбор гиперпараметров

## Предсказание на тестовых данных

Загружаем тестовые данные

In [192]:
test_df = pd.read_csv(TEST)

In [193]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Company               540 non-null    object
 1   Specific Bean Origin  540 non-null    object
 2   REF                   540 non-null    int64 
 3   Review                540 non-null    int64 
 4   Cocoa Percent         540 non-null    object
 5   Company Location      540 non-null    object
 6   Bean Type             540 non-null    object
 7   Broad Bean Origin     540 non-null    object
dtypes: int64(2), object(6)
memory usage: 33.9+ KB


In [194]:
X_test_transformed = clean_df(test_df.copy())

1:  (540, 9)
2:  (540, 10)
3:  (540, 11)
4:  (540, 12)
5:  (540, 12)
6:  (540, 7)


In [196]:
# Заполнение пропусков в Bean Type_cat на основе группировки по Broad Bean Origin_cat и Specific Bean Origin_cat
X_test_transformed['Bean Type_cat'] = X_test_transformed.groupby(['Broad Bean Origin_cat', 'Specific Bean Origin_cat'])['Bean Type_cat'].transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
)

X_test_transformed['Bean Type_cat'] = X_test_transformed.groupby(['Broad Bean Origin_cat'])['Bean Type_cat'].transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
)

X_test_transformed['Bean Type_cat'] = X_test_transformed.groupby(['Specific Bean Origin_cat'])['Bean Type_cat'].transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
)

X_test_transformed['Bean Type_cat'].fillna('unknown', inplace=True)

/var/folders/zc/p2mfk5gx5f1cf_g75s52h3940000gn/T/ipykernel_9158/1383606754.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
/var/folders/zc/p2mfk5gx5f1cf_g75s52h3940000gn/T/ipykernel_9158/1383606754.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
/var/folders/zc/p2mfk5gx5f1cf_g75s52h3940000gn/T/ipykernel_9158/1383606754.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is depre

In [199]:
# Применяем альтернативный вариант
X_test_transformed['Broad Bean Origin_cat'] = impute_by_group_fillna(
    X_test_transformed, 
    'Specific Bean Origin_cat', 
    'Broad Bean Origin_cat'
)

# Заполнение всех пустых значений Broad Bean Origin_cat значением Specific Bean Origin_cat
X_test_transformed['Broad Bean Origin_cat'].fillna(X_test_transformed['Specific Bean Origin_cat'], inplace=True)


/var/folders/zc/p2mfk5gx5f1cf_g75s52h3940000gn/T/ipykernel_9158/4033484276.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
/var/folders/zc/p2mfk5gx5f1cf_g75s52h3940000gn/T/ipykernel_9158/3386355281.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original

In [200]:
X_test_transformed.isna().sum()

Company                     0
Review                      0
Company Location            0
Cocoa Percent_numeric       0
Broad Bean Origin_cat       0
Specific Bean Origin_cat    0
Bean Type_cat               0
dtype: int64

Делаем предсказание

In [202]:
pred = model.predict(X_test_transformed)

Сохраняем предсказание в файл в нужном формате

In [203]:
test_df['id'] = np.arange(len(test_df))
test_df['Rating'] = pred

test_df[['id','Rating']].to_csv("submission_3.csv", index=False)